In [193]:
from pytorch_tdnn.tdnn import TDNN as TDNNLayer
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import math, random

class TDNNv1(torch.nn.Module):

    def __init__(self):
        super(TDNNv1, self).__init__()

        self.tdnn1 = TDNNLayer(16, 8, [-1,0,1])
        self.tdnn2 = TDNNLayer(8, 3, [-2,0,2])
        self.flatten = nn.Flatten()
#         self.conv = torch.nn.Conv1d(n, n, 9, stride=9) # linear layer
        self.linear = nn.Linear(27, 3)
        self.network = nn.Sequential(
            self.tdnn1,
            self.tdnn2,
            self.flatten,
            self.linear,
        )

    def forward(self, x):
        out = self.network(x)
        return out

In [194]:
# note: changes /usr/local/lib/python3.10/site-packages/pytorch_tdnn
# line 41: padding = 0

In [195]:
# # testing on random numbers
# import random
# import numpy as np
# input1 = []
# for i in range(50):
#     curr = []
#     for j in range(16):
#         currow = []
#         for j in range(15):
#             currow.append(random.uniform(0, 1))
#         curr.append(currow)
#     input1.append(curr)
# # input1 = np.array(input1)
# # input1 = np.zeros((16, 16, 16))
# input1 = torch.Tensor(input1)
# tdnn = TDNNv1()
# print(tdnn(input1))

In [196]:
# load dataset
import os, re

x = [] #data
y = [] #labels, where 0 = b, 1 = d, 2 = g

def load_file(f, label, letter):
    fs = f.split("-") # split file name to get starting frame
    start = int(fs[1].split('.')[0])//10
    with open("test2/spectrograms/"+letter+"/"+f, 'r') as g:
        mel_spectrogram = [[float(num) for num in line.split(',')][start:start + 15] for line in g]
        x.append(mel_spectrogram)
    g.close()
    y.append(label)
    
    
for f in sorted(os.listdir("test2/spectrograms/b")):
    load_file(f, 0, "b")
    
for f in sorted(os.listdir("test2/spectrograms/d")):
    load_file(f, 1, "d")
    
for f in sorted(os.listdir("test2/spectrograms/g")):
    load_file(f, 2, "g")
    
# shuffler
# temp = list(zip(x, y))
# random.shuffle(temp)
# x, y = zip(*temp)

# torch dataset, shuffle option

x = torch.Tensor(x)
y = torch.tensor(y, dtype=torch.long)

### DEBUGGING HACKS
# y[5] = 2
x[4:9,:,:] += 0.005;

print(x.shape)
print(y)

torch.Size([54, 16, 15])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2])


In [197]:
# b: ball, bane, bee, bird, bow, bread
# d: dance, date, dive, dog, door, duck
# g: game, gap, gift, go, grow, guide

In [200]:
tdnn = TDNNv1()
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(tdnn.parameters(), lr=0.01, momentum=0.2)

for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
#     for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
    inputs, labels = x*100, y # better var names pls

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = tdnn(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # print statistics
    running_loss += loss.item()
#     if epoch % 5 == 0:  
    print(f'{epoch + 1} loss: {loss.item()/len(y)}')
    running_loss = 0.0

print('Finished Training')

1 loss: 0.020466588161609792
2 loss: 0.020463479889763728
3 loss: 0.020459777779049344
4 loss: 0.020455993987895823
5 loss: 0.020452227857377794
6 loss: 0.020448497048130742
7 loss: 0.020444801560154668
8 loss: 0.020441143601029006
9 loss: 0.020437525378333196
10 loss: 0.020433942476908367
Finished Training


In [201]:
print(outputs)
# for i, output in enumerate(outputs):
#     print(output, y[i])

tensor([[ 0.0452, -0.0127, -0.2292],
        [ 0.0446, -0.0136, -0.2285],
        [ 0.0454, -0.0125, -0.2291],
        [ 0.0476, -0.0113, -0.2314],
        [ 0.0725, -0.0108, -0.2518],
        [ 0.0707, -0.0011, -0.2595],
        [ 0.0644, -0.0054, -0.2540],
        [ 0.0653, -0.0069, -0.2529],
        [ 0.0635, -0.0056, -0.2533],
        [ 0.0466, -0.0126, -0.2291],
        [ 0.0457, -0.0123, -0.2291],
        [ 0.0476, -0.0130, -0.2294],
        [ 0.0531, -0.0095, -0.2312],
        [ 0.0506, -0.0143, -0.2289],
        [ 0.0525, -0.0072, -0.2350],
        [ 0.0464, -0.0125, -0.2290],
        [ 0.0492, -0.0138, -0.2287],
        [ 0.0468, -0.0118, -0.2302],
        [ 0.0470, -0.0120, -0.2289],
        [ 0.0456, -0.0124, -0.2291],
        [ 0.0458, -0.0125, -0.2288],
        [ 0.0455, -0.0122, -0.2292],
        [ 0.0456, -0.0122, -0.2292],
        [ 0.0455, -0.0122, -0.2292],
        [ 0.0453, -0.0130, -0.2286],
        [ 0.0462, -0.0124, -0.2289],
        [ 0.0463, -0.0124, -0.2290],
 

In [144]:
labels

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2])

In [170]:
tdnn2 = TDNNv1()

In [177]:
a = list(tdnn2.modules())[-1]

In [180]:
list(a.parameters())

[Parameter containing:
 tensor([[-0.1214, -0.1550, -0.0239, -0.1863, -0.1289,  0.1869,  0.1455, -0.0934,
          -0.0551,  0.0348, -0.1222, -0.0426, -0.1587,  0.1914,  0.1109,  0.1852,
          -0.1136, -0.0650, -0.1571,  0.0532, -0.0046,  0.0065, -0.1170, -0.1735,
          -0.1010, -0.1023,  0.1757],
         [-0.0281, -0.0491,  0.0910,  0.0473, -0.1303, -0.1823, -0.1140, -0.1902,
          -0.0226, -0.1344,  0.0677, -0.0965,  0.1575,  0.1086, -0.1521, -0.1189,
          -0.1231,  0.0541, -0.0766,  0.0967,  0.0276, -0.1307,  0.1699, -0.1828,
           0.1793, -0.0313,  0.1038],
         [-0.1015,  0.0856, -0.1131, -0.0208,  0.0527, -0.0207, -0.0753,  0.1263,
          -0.0516,  0.0852, -0.1392, -0.1551,  0.0846,  0.1442,  0.0244,  0.1113,
          -0.0886, -0.1547,  0.1403,  0.0623, -0.0596,  0.0397, -0.1175, -0.0737,
          -0.1077, -0.0936, -0.0662]], requires_grad=True),
 Parameter containing:
 tensor([-0.1732,  0.0502, -0.1708], requires_grad=True)]

In [190]:
p = list(a.parameters())

In [191]:
list(list(tdnn.modules())[-1].parameters())

[Parameter containing:
 tensor([[-0.1201, -0.1539, -0.0229, -0.1853, -0.1279,  0.1879,  0.1465, -0.0924,
          -0.0541,  0.0340, -0.1228, -0.0432, -0.1594,  0.1908,  0.1103,  0.1846,
          -0.1143, -0.0657, -0.1583,  0.0520, -0.0057,  0.0053, -0.1182, -0.1747,
          -0.1022, -0.1034,  0.1746],
         [-0.0281, -0.0490,  0.0910,  0.0474, -0.1302, -0.1822, -0.1139, -0.1901,
          -0.0225, -0.1335,  0.0686, -0.0956,  0.1584,  0.1095, -0.1512, -0.1180,
          -0.1222,  0.0550, -0.0758,  0.0974,  0.0284, -0.1299,  0.1708, -0.1820,
           0.1801, -0.0305,  0.1046],
         [-0.1027,  0.0845, -0.1142, -0.0219,  0.0515, -0.0218, -0.0764,  0.1251,
          -0.0527,  0.0850, -0.1394, -0.1554,  0.0844,  0.1440,  0.0241,  0.1110,
          -0.0889, -0.1550,  0.1406,  0.0626, -0.0592,  0.0400, -0.1171, -0.0734,
          -0.1074, -0.0933, -0.0658]], requires_grad=True),
 Parameter containing:
 tensor([-0.1699,  0.0426, -0.1665], requires_grad=True)]

In [192]:
p

[Parameter containing:
 tensor([[-0.1201, -0.1539, -0.0229, -0.1853, -0.1279,  0.1879,  0.1465, -0.0924,
          -0.0541,  0.0340, -0.1228, -0.0432, -0.1594,  0.1908,  0.1103,  0.1846,
          -0.1143, -0.0657, -0.1583,  0.0520, -0.0057,  0.0053, -0.1182, -0.1747,
          -0.1022, -0.1034,  0.1746],
         [-0.0281, -0.0490,  0.0910,  0.0474, -0.1302, -0.1822, -0.1139, -0.1901,
          -0.0225, -0.1335,  0.0686, -0.0956,  0.1584,  0.1095, -0.1512, -0.1180,
          -0.1222,  0.0550, -0.0758,  0.0974,  0.0284, -0.1299,  0.1708, -0.1820,
           0.1801, -0.0305,  0.1046],
         [-0.1027,  0.0845, -0.1142, -0.0219,  0.0515, -0.0218, -0.0764,  0.1251,
          -0.0527,  0.0850, -0.1394, -0.1554,  0.0844,  0.1440,  0.0241,  0.1110,
          -0.0889, -0.1550,  0.1406,  0.0626, -0.0592,  0.0400, -0.1171, -0.0734,
          -0.1074, -0.0933, -0.0658]], requires_grad=True),
 Parameter containing:
 tensor([-0.1699,  0.0426, -0.1665], requires_grad=True)]